In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import os


llm = ChatOpenAI(base_url='https://api.deepseek.com', api_key=os.getenv('DEEPSEEK_SK'), model='deepseek-chat')

prompt_extract = ChatPromptTemplate.from_template(
    "请从以下文本中提取技术规格：\n\n{text_input}"
)

prompt_transformer = ChatPromptTemplate.from_template(
    "请将以下技术规格转为JSON格式，包含'cpu'、'memory'、'storage'三个键: \n\n{specifications}"
)

# 用 LCEL 构建链
# # StrOutputParser() 将 LLM 消息输出转为字符串
extraction_chain = prompt_extract | llm | StrOutputParser()

full_chain = (
    {"specifications": extraction_chain}
    | prompt_transformer | llm | StrOutputParser()
)

# 运行链
input_text = "新款笔记本配备 3.5GHz 八核处理器、16GB 内存和 1TB NVMe SSD。"

# 执行链
final_result = full_chain.invoke({"text_input": input_text})

print('-----输出结果-----')
print(final_result)


-----输出结果-----
{
    "cpu": "3.5GHz 八核处理器",
    "memory": "16GB",
    "storage": "1TB NVMe SSD"
}
